# Atividade • Data Engineering
<hr>

## Data Discovery

### Formato

```%h %l %u %t \"%r\" %>s %b \"%{Referer}i\" \"%{User-agent}i\```

### Dicionário

<p> <strong>%h</strong>: Host remoto (endereço IP do cliente) </p>
<p> <strong>%l</strong>: Identidade do usuário, ou traço, se não houver (geralmente não é usado)</p>
<p> <strong>%u</strong>: Nome de usuário, via autenticação HTTP ou traço se não for usado</p>
<p> <strong>%t</strong>: Carimbo de data e hora de quando o Apache recebeu a solicitação HTTP </p>
<p> <strong>\”%r\</strong>: A própria solicitação real do cliente </p>
<p> <strong>%>s</strong>: O código de status que o Apache retorna em resposta à solicitação </p>
<p> <strong>%T</strong>: O tempo de resposta em milissegundos. </p>
<p> <strong>\”%{Referer}i\”</strong>: Cabeçalho do referenciador ou travessão, se não for usado (em outras palavras, eles clicaram em um URL de outro site para acessar o seu site) </p>
<p> <strong>\”%{User-agent}i\</strong>: User agent (contém informações sobre o navegador/SO/etc do solicitante) </p>

## Objetivos

### Explorar as habilidades e o conhecimento sobre: desenvolvimento de scripts, tratamento de dados, estrutura de dados, ordenação, parse e formatação.

<p> <strong>1.</strong> Formatar uma saída do log em json contendo a lista de request apresentada no log, cada objeto dentro da lista deve conter as propriedades de uma entrada no log como remote_host, date, request, status_code, response_time, reffer, user_agent; </p>
<p> <strong>2.</strong> Encontrar os 10 maiores tempos de resposta com sucesso do servidor na chamada GET /maunal/ com a origem do tráfego (reffer) igual a <strong><a href="http://localhost/svnview?repos=devel&rev=latest&root=SVNview/tmpl&list_revs=1">http://localhost/svnview?repos=devel&rev=latest&root=SVNview/tmpl&list_revs=1</a></strong>; </p>
<p> <strong>3.</strong> Formatar uma saída em arquivo físico do access.log com a data em formato UNIX timestamp %Y-%m-%d %H:%M:%S e o IP convertido em um hash MD5; </p>
<p> <strong>4.</strong> Formatar uma saída em arquivo físico agrupando a soma total de requests por dia do ano; </p>
<p> <strong>5.</strong> Formatar uma saída em arquivo físico com endereços de IP únicos, um IP por linha, contidos no log com a última data de request realizado pelo remote IP. </p>

## Preparando o Ambiente

### Distribuição Linux

In [1]:
! lsb_release -a

No LSB modules are available.
Distributor ID:	Ubuntu
Description:	Ubuntu 22.04.3 LTS
Release:	22.04
Codename:	jammy


### Versão do Python

In [2]:
! python -V

Python 3.10.12


### Instalando as libs

In [3]:
! pip install -r "../requirements.txt"

### Instalando o JDK

- Execute no terminal o seguinte comando:
  - ```sudo apt-get install openjdk-8-jdk-headless -qq > /dev/null```

In [4]:
! java -version

openjdk version "1.8.0_382"
OpenJDK Runtime Environment (build 1.8.0_382-8u382-ga-1~22.04.1-b05)
OpenJDK 64-Bit Server VM (build 25.382-b05, mixed mode)


### Download do Spark

In [ ]:
! wget -q -P "../spark" "https://archive.apache.org/dist/spark/spark-3.5.0/spark-3.5.0-bin-hadoop3.tgz"

In [ ]:
! tar xf "../spark/spark-3.5.0-bin-hadoop3.tgz" -C "../spark"

In [ ]:
! rm -rf "../spark/spark-3.5.0-bin-hadoop3.tgz"

### Importando as bibliotecas

In [5]:
from pyspark.sql import SparkSession
from pyspark.sql.types import DoubleType, IntegerType, FloatType
from pyspark.sql import functions as f

### Instanciando o Spark

In [6]:
! export JAVA_HOME=$(readlink -f $(which java))
! export SPARK_HOME="$(readlink -f ../)/spark/spark-3.5.0-bin-hadoop3"

In [7]:
import findspark
findspark.init()

In [8]:
spark = SparkSession.builder\
                    .appName("Atividade Data Engineering - Multiplan")\
                    .getOrCreate()

your 131072x1 screen size is bogus. expect trouble
23/10/24 15:26:39 WARN Utils: Your hostname, DESKTOP-FLROA7K resolves to a loopback address: 127.0.1.1; using 172.23.17.233 instead (on interface eth0)
23/10/24 15:26:40 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/10/24 15:26:41 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [9]:
spark

## Lendo o arquivo

In [10]:
df = spark.read.text('../data/raw/test-access-001-1.log')

In [12]:
df.show(truncate=False)

+--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|value                                                                                                                                                                                                                                                                                                                                                                     |
+-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------